In [21]:
import cv2
import time
import mediapipe as mp

import numpy as np 
import joblib
import tensorflow as tf

from apscheduler.schedulers.background import BackgroundScheduler


In [34]:
def get_valid_landmarks(hand_landmarks, image_width, image_height):
    valid_landmarks = []
    for landmark in hand_landmarks.landmark:
        x = int(landmark.x * image_width)
        y = int(landmark.y * image_height)
        
        if 0 <= x < image_width and 0 <= y < image_height:
            valid_landmarks.append((x, y))
    
    return valid_landmarks

def initialize_hand_model():
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    return hands, mp_drawing, mp_hands

def calculate_fps(prev_time):
    curr_time = time.time()
    fps = 1 / (curr_time - prev_time)
    return fps, curr_time

def extract_landmark(hand_landmarks):
    partial_landmarks = []
    for landmark in hand_landmarks.landmark:
        partial_landmarks.append(landmark.x)
        partial_landmarks.append(landmark.y)
        partial_landmarks.append(landmark.z)    
    # print('extract_landmark: ', partial_landmarks)
    return partial_landmarks

def predict_model_single(landmark_data):
    global class_labels
    
    scaler = joblib.load("scaler.pkl")
    landmark_data = np.array(landmark_data).reshape(1, -1)
    landmark_data = scaler.transform(landmark_data)
    
    model = tf.keras.models.load_model('./models/14-juni-1058am.h5')
    prediction = model.predict(landmark_data)
    prediction = np.argmax(prediction, axis=1)
    prediction = class_labels[prediction[0]]
    
    return prediction

def batch_prediction(landmarks_data):
    global class_labels
    results = []
    scaler = joblib.load("scaler.pkl")
    model = tf.keras.models.load_model('./models/14-juni-1058am.h5')
    
    for landmark_data in landmarks_data:
        landmark_data = np.array(landmark_data).reshape(1, -1)
        landmark_data = scaler.transform(landmark_data)
        prediction = model.predict(landmark_data)
        prediction = np.argmax(prediction, axis=1)
        prediction = class_labels[prediction[0]]
        results.append(prediction)
    return results
        


def display_hand_status(frame, results, mp_drawing, mp_hands, image_width, image_height):
    global landmarks_buffer
    total_hands = 0
    status_text = ""
    
    # result_predict = ""
    if results.multi_hand_landmarks:
        landmarks = []
        for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
            total_hands += 1
            
            valid_landmarks = get_valid_landmarks(hand_landmarks, image_width, image_height)
            num_landmarks = len(valid_landmarks)
            
            if num_landmarks < 21:
                None
                # print(f'{idx+1} | {num_landmarks} Tangan ketutupan')
                # print(f'{idx+1} | {num_landmarks} Tangan ketutupan')
            elif num_landmarks == 21:
                landmarks.extend(extract_landmark(hand_landmarks))
                
            
            status = "Valid" if num_landmarks == 21 else "Not Valid"
            status_text += f"h{idx+1}_stats: {status}, h{idx+1}_lmk: {num_landmarks}\n"

            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        if landmarks != None and len(landmarks) == 63:
            landmarks_buffer.append(landmarks)
            # print(landmarks)
            # print(len(landmarks))
            # result_predict = predict_model_single(landmarks)
                
            
        
    status_text = f"hand detected: {total_hands}\n" + status_text.strip()
    y0, dy = 50, 30
    
    for i, line in enumerate(status_text.split('\n')):
        y = y0 + i * dy
        cv2.putText(frame, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)
    
    # if result_predict != "":
    #     text_size = cv2.getTextSize(result_predict, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
    #     text_x = (image_width - text_size[0]) // 2
    #     text_y = image_height - 20  # 20 pixels from the bottom
    #     cv2.putText(frame, result_predict, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
    return frame


In [16]:
cv2.destroyAllWindows()

In [35]:
class_labels = ['A', 'Aku', 'Apa', 'B', 'Bagaimana', 'Baik', 'Bapak', 'Berapa',
    'Besok', 'C', 'D', 'Dia', 'Dimana', 'E', 'F', 'G', 'H', 'Halo',
    'Hari ini', 'I', 'Ibu', 'J', 'Jawab', 'K', 'Kalian', 'Kamu',
    'Kantor', 'Kapan', 'Kemana', 'Kemarin', 'Kerja', 'L', 'Lelah',
    'Lusa', 'M', 'Maaf', 'Makan', 'Malam', 'Mengapa', 'N', 'Nanti',
    'O', 'P', 'Pagi', 'Q', 'R', 'S', 'Sabar', 'Sakit', 'Sama - sama',
    'Sedih', 'Sekarang', 'Senang', 'Siang', 'Siapa', 'Sore', 'T',
    'Terima kasih', 'Tidur', 'Tolong', 'U', 'V', 'W', 'X', 'Y', 'Z']

landmarks_buffer = []
last_prediction = ""

def main():
    global landmarks_buffer, last_prediction
    hands, mp_drawing, mp_hands = initialize_hand_model()

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Tidak dapat mengakses kamera.")
        return

    prev_time = 0
    # landmarks_buffer = []

    def scheduled_prediction():
        global last_prediction
        if landmarks_buffer:
            print(len(landmarks_buffer))
            print(f"sample: {landmarks_buffer[0]}")
            result_predict = batch_prediction(landmarks_buffer)
            if result_predict:
                last_prediction = result_predict[0]
            landmarks_buffer.clear()

    scheduler = BackgroundScheduler()
    scheduler.add_job(scheduled_prediction, 'interval', seconds=3)
    scheduler.start()
    
    try:
        while True:
            ret, frame = cap.read()
            
            if not ret:
                print("Error: Tidak dapat membaca frame dari kamera.")
                break

            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            
            image_height, image_width, _ = frame.shape
                                
            frame = display_hand_status(frame, results, mp_drawing, mp_hands, image_width, image_height)
            fps, prev_time = calculate_fps(prev_time)
            
            # global landmarks_buffer
            # print(len(landmarks_buffer))
            
            cv2.putText(frame, f'FPS: {int(fps)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, last_prediction, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imshow('Cam', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # print(len(landmarks_buffer))
        # print(landmarks_buffer[0])
        # print(len(landmarks_buffer[0]))
        # result_predict = batch_prediction(landmarks_buffer)
        
        # if len(result_predict) >= 0 :
        #     text_size = cv2.getTextSize(result_predict, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)[0]
        #     text_x = (image_width - text_size[0]) // 2
        #     text_y = image_height - 20  # 20 pixels from the bottom
        #     cv2.putText(frame, result_predict, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    finally:
        cap.release()
        cv2.destroyAllWindows()
        scheduler.shutdown()

if __name__ == "__main__":
    main()

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


80
sample: [0.16644741594791412, 0.9996970295906067, 6.896043487358838e-07, 0.24819977581501007, 0.9460670948028564, -0.0306857880204916, 0.3191412091255188, 0.8493561744689941, -0.04869457334280014, 0.3663724362850189, 0.7633040547370911, -0.06625044345855713, 0.4128212332725525, 0.6966322064399719, -0.0844246968626976, 0.27168959379196167, 0.6955568790435791, -0.03209220618009567, 0.3255346417427063, 0.6017340421676636, -0.07047801464796066, 0.36501404643058777, 0.5536246299743652, -0.0993872806429863, 0.40543293952941895, 0.5187636017799377, -0.11881130188703537, 0.21817301213741302, 0.6695007085800171, -0.0428316630423069, 0.2481027990579605, 0.540604293346405, -0.07709039002656937, 0.281445175409317, 0.46267247200012207, -0.10090214014053345, 0.3145278990268707, 0.39827659726142883, -0.11663536727428436, 0.16168183088302612, 0.6857743263244629, -0.059444110840559006, 0.16413623094558716, 0.5474374890327454, -0.09808888286352158, 0.1800146847963333, 0.4552145302295685, -0.123428367

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\skl

1/1 [==============================] - 0s 17ms/step


c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
 

...

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - ETA: 0s

c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 24ms/step


c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\whisn\anaconda3\envs\bangkitcourse\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
